# Excercises 

## 0. Setup your own repo
- Setting up a repo is often repetitive. That's why you can use what are called 'cookiecutters', providing you with a template structure with some basic folders and files already set up for you. You don't have to use one, and can do it by hand, but have a look and maybe you think it is helpful.
    - You could use the `cookiecutter` command that is preinstalled on your VM to create a repo, see https://github.com/raoulg/datascience-cookiecutter for details. I made this one myself, because I ended up editing the cookiecutter I was using.
    - Another project is this one https://github.com/drivendata/cookiecutter-data-science , also intended for data science projects
    - more general cookiecutters are shipped with tools like [pdm](https://pdm-project.org/latest/) and [rye](https://rye-up.com/); starting a project with `pdm init` or `rye init` (see docs for details) will provide you with some minimal structure, and there is the option to provide your own template with pdm (see [pdm template](https://pdm-project.org/latest/usage/template/))
- push your own repo to github. Use `MADS-ML-{yourname}` as a format, eg `MADS-ML-JoostB`.
- Invite me (raoulg; https://github.com/raoulg) as a collaborator to your repo.
- make the excercises 1-5 below in your repo, and push them to github.

Tips:
- Commit often (every 30 minutes or so) 
- really, commit often. commiting and pushing your work is the best way to make sure your work is saved properly.
- Commit groups of files that are related to each other. If you have more files, commit them separately.
- Write commit messages that are descriptive and informative. "lesson 1", "changes" or "commit" are bad commit messages; "added excercise 2" is better, "[exercise 2] added __len__ to Dataset class" is even better.
- Use `pdm` or `rye` to add dependencies. `mads_datasets` and `mltrainer` should cover a lot of what you need; don't blindly copy-paste all dependencies but keep your `pyproject.toml` as clean as possible.

At some point, you will get a grade for the excercises that is 0 (not good enough), 1 (good enough) or 2 (excellent).
I will look for both form and correctness to determine your grade.
The result be incorporated into your final grade for this course.

## 1. 3D Tensor dataset
- Create a random 3D tensor dataset with `torch`
- Build your own `DataSet` class, such that you can get a 3D tensor and a label (which can be a random 0 or 1)
See notebook 03_dataloader for details on how to create a custom dataset. See 01_tensors and the torch documentation how to create random tensors.

## 2. Datastreamers
Study the `BaseDatastreamer` in `03_dataloader` and use it with your own dataset, such that you get a datastreamer that will keep on giving you new batches of data when you call `next()` or loop over it.

# 3. Tune the network
For this exercise we won't build upon the previous exercises, but instead will use the Fashion dataset.
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 

In [39]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
import gin

In [40]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

In [41]:
preprocessor = BasePreprocessor()
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2024-12-07 20:48:42.696 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2024-12-07 20:48:42.697 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt
/home/azureuser/MachineLearning/.venv/lib/python3.11/site-packages/mads_datasets/factories/torchfactories.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

In [42]:
print(gin.config_str())

import gin.torch.external_configurables

# Parameters for NeuralNetwork:
# ==============================================================================
NeuralNetwork.num_classes = 10
NeuralNetwork.units1 = 512
NeuralNetwork.units2 = 64



A big advantage is that we can save this config as a file; that way it is easy to track what you changed during your experiments.

In [43]:
accuracy = metrics.Accuracy()

Or, you can use gin, it will read the model.gin file, and instead of needing to set

You can gin.parce_config_file('model.gin') and then set the model with model = NeuralNetwork(), and the parameters will be loaded from the gin file.

If you want to combine this with a manual grid search, you could automate that with a double forloop:

In [44]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 128
Units: 128, 64
Units: 64, 64


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

!!! Train_steps en valid_steps is verkleind om de runtime op de vm werkbaar te houden. Dit is gezet op 64. Dus 64 batches.

!!!! Epochs vergroot naar 25 om te zien wanneer er een kantelpunt is. Wanneer ga je overfitten?

!!!! De resultaten met unit2 = 64 hebben de slechtste resultaten.

In [45]:
import torch
gin.parse_config_file("model.gin")

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=50,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=64,
    valid_steps=64,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        gin.bind_parameter("NeuralNetwork.units1", unit1)
        gin.bind_parameter("NeuralNetwork.units2", unit2)

        model = imagemodels.NeuralNetwork()
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.SGD,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()

2024-12-07 20:49:49.645 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellogs/20241207-204949
2024-12-07 20:49:49.646 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 64/64 [00:00<00:00, 285.54it/s]
2024-12-07 20:49:50.039 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 2.2994 test 2.2964 metric ['0.1650']
100%|██████████| 64/64 [00:00<00:00, 295.07it/s]
2024-12-07 20:49:50.430 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 2.2927 test 2.2894 metric ['0.2166']
100%|██████████| 64/64 [00:00<00:00, 297.12it/s]
2024-12-07 20:49:50.805 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 2.2862 test 2.2828 metric ['0.2649']
100%|██████████| 64/64 [00:00<00:00, 306.81it/s]
2024-12-07 20:49:51.171 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 2.2786 test 2.2722 metric ['0.3533']
100%|██████████| 64/64 [00:00<00:00, 301.33it/s]
2024-12-07 20:49

In [46]:
settings

epochs: 50
metrics: [Accuracy]
logdir: modellogs
train_steps: 64
valid_steps: 64
reporttypes: [<ReportTypes.TENSORBOARD: 2>, <ReportTypes.GIN: 1>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.1, 'patience': 10}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 10}

In [47]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

!!! Er zijn 10 verschillende fashion categorien. Dus out_features = 10

Experiment 2:
andere optimizer : SGD => Adam

In [48]:
import torch
gin.parse_config_file("model.gin")

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=50,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=64,
    valid_steps=64,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        gin.bind_parameter("NeuralNetwork.units1", unit1)
        gin.bind_parameter("NeuralNetwork.units2", unit2)

        model = imagemodels.NeuralNetwork()
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()

2024-12-07 20:52:39.362 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellogs/20241207-205239
2024-12-07 20:52:39.363 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 64/64 [00:00<00:00, 228.77it/s]
2024-12-07 20:52:39.812 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 1.0655 test 0.7377 metric ['0.7256']
100%|██████████| 64/64 [00:00<00:00, 238.81it/s]
2024-12-07 20:52:40.243 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.6626 test 0.6256 metric ['0.7708']
100%|██████████| 64/64 [00:00<00:00, 235.44it/s]
2024-12-07 20:52:40.680 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.5554 test 0.5344 metric ['0.8088']
100%|██████████| 64/64 [00:00<00:00, 237.52it/s]
2024-12-07 20:52:41.115 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 0.5145 test 0.5507 metric ['0.7981']
2024-12-07 20:52:41.116 | INFO     | mltrainer.trainer:__call__:2

!!!!!! Optimizer Adam presteert veel beter.

Experiment 3: optimizer Adam => AdamW

In [49]:
import torch
gin.parse_config_file("model.gin")

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=50,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=64,
    valid_steps=64,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        gin.bind_parameter("NeuralNetwork.units1", unit1)
        gin.bind_parameter("NeuralNetwork.units2", unit2)

        model = imagemodels.NeuralNetwork()
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.AdamW,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()

2024-12-07 20:55:31.607 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellogs/20241207-205531
2024-12-07 20:55:31.608 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 64/64 [00:00<00:00, 250.25it/s]
2024-12-07 20:55:32.022 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 1.0774 test 0.6732 metric ['0.7437']
100%|██████████| 64/64 [00:00<00:00, 235.54it/s]
2024-12-07 20:55:32.452 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.6372 test 0.5773 metric ['0.7798']
100%|██████████| 64/64 [00:00<00:00, 251.16it/s]
2024-12-07 20:55:32.874 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.5514 test 0.5664 metric ['0.8005']
100%|██████████| 64/64 [00:00<00:00, 251.94it/s]
2024-12-07 20:55:33.295 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 0.5462 test 0.5865 metric ['0.7693']
2024-12-07 20:55:33.296 | INFO     | mltrainer.trainer:__call__:2

!!! Verschillen tussen Adam en AdamW zijn klein.

Run the experiment, and study the result with tensorboard.

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

in the terminal, cd to the location of the repository
activate the python environment for the shell. Note how the correct environment is being activated.
run tensorboard --logdir=modellogs in the terminal
tensorboard will launch at localhost:6006 and vscode will notify you that the port is forwarded
you can either press the launch button in VScode or open your local browser at localhost:6006

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`


Experiment with things like:

- changing the amount of units1 and units2 to values between 16 and 1024. Use factors of 2: 16, 32, 64, etc.
- changing the batchsize to values between 4 and 128. Again, use factors of two.
- all your experiments are saved in the `modellogs` directory, with a timestamp. Inside you find a saved_config.gin file, that 
contains all the settings for that experiment. The `events` file is what tensorboard will show.
- plot the result in a heatmap: units vs batchsize.
- changing the learningrate to values between 1e-2 and 1e-5 
- changing the optimizer from SGD to one of the other available algoritms at [torch](https://pytorch.org/docs/stable/optim.html) (scroll down for the algorithms)

A note on train_steps: this is a setting that determines how often you get an update. 
Because our complete dataset is 938 (60000 / 64) batches long, you will need 938 trainstep to cover the complete 60.000 images.

This can actually be a bit confusion, because every value below 938 changes the meaning of `epoch` slightly, because one epoch is no longer
the full dataset, but simply `trainstep` batches. Setting trainsteps to 100 means you need to wait twice as long before you get feedback on the performance,
as compared to trainsteps=50. You will also see that settings trainsteps to 100 improves the learning, but that is simply because the model has seen twice as 
much examples as compared to trainsteps=50.

This implies that it is not usefull to compare trainsteps=50 and trainsteps=100, because setting it to 100 will always be better.
Just pick an amount, and adjust your number of epochs accordingly.